# Dust Attenuation Curves

We implment a whole suite of different attenuation curves in Synthesizer, including the following:

- ``PowerLaw``: A power-law attenuation curve.
- ``Calzetti2000``: The [Calzetti (Calzetti et al. 2000)](https://ui.adsabs.harvard.edu/abs/2000ApJ...533..682C/abstract) attenuation curve (with an optional UV bump from [Noll et al. 2009](https://ui.adsabs.harvard.edu/abs/2009A%26A...507.1793N/abstract)).
- ``MWN18``: A Milky Way attenuation curve using in [Narayanan et al. 2018](https://ui.adsabs.harvard.edu/abs/2018ApJ...869...70N/abstract).
- ``GrainsWD01``: A dust grain attenuation curve from [Weingarter and Draine 2001](https://ui.adsabs.harvard.edu/abs/2001ApJ...548..296W/abstract) with models for the Milky Way, LMC, and SMC (and more defined in WD01).
- ``ParametricLi08``: A parametric and empirically derived attenuation curve implemented in [Li et al. 2008](https://ui.adsabs.harvard.edu/abs/2008arXiv0808.4123L/abstract) and including parameters from [Markov et al. 2023](https://ui.adsabs.harvard.edu/abs/2023A%26A...679A..12M/abstract) and [Markov et al. 2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240205996M/abstract) extending it to $z=12$.

These attenuation curves can be instatiated directly to compare transmission curves or attached to an [``EmissionModel``](../emission_models/emission_models.rst) to be used in the generation of complex spectra from a galaxy or galaxy component.

Each of these models has unique arguments needed to instantiate them but they are have the exact same ``get_transmission`` method which requires an optical depth (``tau_v``) and a wavelength array. Below, we show how to instantiate each of these models and plot them.

To evaluate the transmission curve, we are going to need a wavelength array.

In [ ]:
import numpy as np

lams = np.logspace(2, 4, 1000)


## PowerLaw

A ``PowerLaw`` only requires a slope to be defined.

In [ ]:
from synthesizer.emission_models.attenuation import PowerLaw

power_law = PowerLaw(-1.0)

pl_trans = power_law.get_transmission(0.33, lams)

We can also plot the transmission curve.

In [ ]:
power_law.plot_transmission(0.33, lams, show=True)

# Calzetti2000

The ``Calzetti2000`` model requires a slope (``slope``), central wavelength of the UV bump (``cent_lam``), an amplitude of the UV bump (``ampl``), and the FWHM of the UV bump (``gamma``). These default to 0.0, 0.2175 microns, 0, and 0.035 respectively. We'll plot the transmission curve both with the defaults and a non-zero amplitude.

In [ ]:
from synthesizer.emission_models.attenuation import Calzetti2000

no_bump = Calzetti2000()
with_bump = Calzetti2000(ampl=10.0)

nb_trans = no_bump.get_transmission(0.33, lams)
wb_trans = with_bump.get_transmission(0.33, lams)

fig, ax = no_bump.plot_transmission(0.33, lams, show=False, label="No UV bump")
_, _ = with_bump.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=True, label="With UV bump"
)

## MWN18

The ``MWN18`` model loads a data file included with Synthesizer so requires no arguments.

In [ ]:
from synthesizer.emission_models.attenuation import MWN18

mwn18 = MWN18()

mwn_trans = mwn18.get_transmission(0.33, lams)

_, _ = mwn18.plot_transmission(0.33, lams, show=True)

# GrainsWD01

The ``GrainsWD01`` model requires the model to be defined (either ``'MW'``, ``'LMC'``, or ``'SMC'``). This will use the [dust_extinction module](https://dust-extinction.readthedocs.io/en/latest/#) to load the appropriate model.

In [ ]:
from synthesizer.emission_models.attenuation import GrainsWD01

mw = GrainsWD01("MW")
smc = GrainsWD01("SMC")
lmc = GrainsWD01("LMC")

mwn_trans = mw.get_transmission(0.33, lams)
smc_trans = smc.get_transmission(0.33, lams)
lmc_trans = lmc.get_transmission(0.33, lams)

fig, ax = mw.plot_transmission(0.33, lams, show=False, label="MW")
_, _ = smc.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="SMC"
)
_, _ = lmc.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=True, label="LMC"
)

# ParametricLi08

The ``ParametricLi08`` model requires the UV slope (``UV_slope``, default 1.0), the optical to Near Infrared slope (``OPT_NIR_SLOPE``, default 1.0), the Far UV slope (``FUV_slope``, default 1.0), and a dimensionless parameter between 0 and 1 controlling the strength of the UV bump (``bump``, default 0.0). Alternatively, a model string can be passed to adopt a preset model (possible values: ``"MW"``, ``"LMC"``, ``"SMC"``, or ``Calzetti``).

In [ ]:
from synthesizer.emission_models.attenuation import ParametricLi08

mw_li08 = ParametricLi08(model="MW")
lmc_li08 = ParametricLi08(model="LMC")
smc_li08 = ParametricLi08(model="SMC")
calz_li08 = ParametricLi08(model="Calzetti")

mw_trans = mw_li08.get_transmission(0.33, lams)
lmc_trans = lmc_li08.get_transmission(0.33, lams)
smc_trans = smc_li08.get_transmission(0.33, lams)
calz_trans = calz_li08.get_transmission(0.33, lams)

fig, ax = mw_li08.plot_transmission(0.33, lams, show=False, label="MW")
_, _ = lmc_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="LMC"
)
_, _ = smc_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="SMC"
)
_, _ = calz_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=True, label="Calzetti"
)

## Comparing them all

Now that we've covered everything lets compare them all on a single plot.

In [ ]:
fig, ax = power_law.plot_transmission(
    0.33, lams, show=False, label="Power Law"
)
_, _ = no_bump.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="Calzetti (No UV bump)"
)
_, _ = with_bump.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="Calzetti (With UV bump)"
)
_, _ = mwn18.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="MWN18"
)
_, _ = mw.plot_transmission(0.33, lams, fig=fig, ax=ax, show=False, label="MW")
_, _ = lmc.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="LMC"
)
_, _ = smc.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="SMC"
)
_, _ = mw_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="Li08 (MW)"
)
_, _ = lmc_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="Li08 (LMC)"
)
_, _ = smc_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=False, label="Li08 (SMC)"
)
_, _ = calz_li08.plot_transmission(
    0.33, lams, fig=fig, ax=ax, show=True, label="Li08 (Calzetti)"
)